# Interaction with RChain network via Python/Notebooks

## 1. setup

In current docker compose configuration we have the node name in the environments

In [1]:
RNODE_HOST = %env NODE_NAME

Open a channel with `grpc` using our Python library.

In [2]:
from rchain_grpc import casper

connection = casper.create_connection(host=RNODE_HOST)
connection

## 2. simple deploy

Write code in RhoLang

In [3]:
rholang_code = """
new print(`rho:io:stdout`) in {
    print!("Hello World!")
}
"""

You can deploy/propose to the node you're connected with the string containing Rholang code

In [4]:
casper.deploy(connection, rholang_code)

{'success': True, 'message': 'Success!'}

In [5]:
casper.propose(connection)

{'success': True, 'message': 'Success! Block a0317bec23... created and added.'}

## 3. advanced deploy

A more interesting example is to run a contract and listen to the node for an answer on the deployed block.

To make this trick we define a placeholder name that will be used in the final deployed contract to listen and show the content of block we propose.

In [6]:
output_placeholder = "your_output_placeholder"

rholang_code = """
%s!("bar")
""" % output_placeholder

In [7]:
output = casper.run_and_get_value_from(
    connection=connection,
    term=rholang_code, 
    output_placeholder=output_placeholder
)

In [8]:
# verify the status
if output.get('status').lower() == 'success':
    # all is good, pretty print results
    print("Results:")
    from pprint import pprint
    pprint(output.get('blockResults'))
else:
    # uh-oh something went wrong
    print("Failed!")

Results:
[{'block': {'blockHash': '102525680dac7b71c42c8fe7ebbf4aa7e247df6515dd294699490ef689e04258',
            'blockNumber': 2,
            'blockSize': '1144',
            'deployCount': 1,
            'faultTolerance': -1.0,
            'mainParentHash': 'a0317bec23a50598e08ee31153ef686fddec9761a07d480a8f83f5e42e26e40d',
            'parentsHashList': ['a0317bec23a50598e08ee31153ef686fddec9761a07d480a8f83f5e42e26e40d'],
            'sender': 'eabe5a1a0750d2a8745709bb0bdb24f63c6a8ac3a887b9bed40b34b0598ddf08',
            'timestamp': 1540025659499,
            'tupleSpaceHash': '1f77579c6b9214199596ecb79facfe2b8a7eb4dc674cc4b694dc47238023751b'},
  'postBlockData': [['bar']]}]
